# 01. N-gram 언어 모델: 통계로 단어 맞히기

## 1. N-gram의 개념

**언어 모델(Language Model)**의 기초 원리는 **"앞에 나온 단어들을 보고, 그 다음에 나올 단어를 확률적으로 예측하는 것"**입니다.
우리가 자주 사용하는 **스마트폰 키보드 자동완성** 기능이 대표적인 예시입니다.

### 쉬운 비유
마치 **빈칸 채우기** 문제입니다.
- 문제: "오늘 점심은 맛있는 [ ? ]"
- 예측 후보: 밥(80%), 거(10%), 비(5%)...

이 중에서 가장 확률이 높은 "밥"을 추천하는 것이 언어 모델의 역할입니다.

### 핵심 용어: N (몇 개를 볼까?)
몇 개의 단어를 한 덩어리로 묶어서 볼 것인지를 결정하는 숫자입니다.
이 숫자를 **N**이라고 부릅니다.

| 이름 | N | 설명 | 구조 시각화 |
|---|---|---|---|
| **Unigram** | 1 | 단어 1개씩 따로 봅니다 | `[I]`, `[love]`, `[NLP]` |
| **Bigram** | 2 | **2개씩** 짝지어 봅니다 | `[I -> love]`, `[love -> NLP]` |
| **Trigram** | 3 | 3개씩 짝지어 봅니다 | `[I, love -> NLP]` |

이 실습에서는 **Bigram (2개씩 짝짓기)** 모델을 만들어 봅니다.
즉, **"바로 앞 단어 1개를 힌트로, 다음 단어를 맞추는 모델"**입니다.

## 2. 도구 준비

언어 처리에 필요한 기본 도구들을 불러옵니다.

- `nltk`: 자연어 처리 도구 모음 (단어를 자르는 칼 포함)
- `ngrams`: 단어들을 N개씩 묶어주는 도구
- `Counter`: 개수를 세어주는 계산기

In [2]:
import nltk
from nltk.util import ngrams      # N-gram 생성 함수
from collections import Counter   # 빈도수 계산 클래스

# NLTK 토크나이저 데이터 다운로드 (최초 1회)
nltk.download('punkt', quiet=True) 
print("라이브러리 로드 완료")

라이브러리 로드 완료


## 3. 데이터 자르기 (Tokenization)

모델을 학습시키기 위한 문장 데이터(Corpus)를 준비합니다.
문장을 단어 단위인 **토큰(Token)**으로 잘게 자릅니다.

In [3]:
# 학습 데이터
text = "오늘은 날씨가 좋다. 오늘은 기분이 좋다. 오늘은 일이 많다. 오늘은 사람이 많다. 오늘은 날씨가 맑다."

# 문장을 단어 단위로 분리
tokens = nltk.word_tokenize(text)

print(f"원본 문장: \"{text}\"")
print(f"자른 결과: {tokens}")

원본 문장: "오늘은 날씨가 좋다. 오늘은 기분이 좋다. 오늘은 일이 많다. 오늘은 사람이 많다. 오늘은 날씨가 맑다."
자른 결과: ['오늘은', '날씨가', '좋다', '.', '오늘은', '기분이', '좋다', '.', '오늘은', '일이', '많다', '.', '오늘은', '사람이', '많다', '.', '오늘은', '날씨가', '맑다', '.']


## 4. 짝꿍 만들기 (Bigram)

토큰들을 2개씩 묶어서 패턴을 찾습니다.

### 구조 시각화
```
원본 데이터: [오늘은, 날씨가, 좋다]

1번 짝꿍:  [오늘은] ---> [날씨가]
2번 짝꿍:            [날씨가] ---> [좋다]
```

이렇게 짝을 지은 뒤, 각 짝이 **몇 번 등장했는지** 세어봅니다.

In [4]:
# 1. Bigram 생성 (2개씩 묶기)
bigram = list(ngrams(tokens, 2))

# 2. 개수 세기 (Counter 활용)
unigram_freq = Counter(tokens)  # 각 단어가 몇 번 나왔나? (분모)
bigram_freq = Counter(bigram)   # 짝꿍 단어가 몇 번 나왔나? (분자)

print("[분석 결과: '오늘은' 뒤에 무엇이 왔을까?]")
for pair, count in bigram_freq.items():
    if pair[0] == '오늘은':
        print(f"  {pair} -> {count}번 등장")

[분석 결과: '오늘은' 뒤에 무엇이 왔을까?]
  ('오늘은', '날씨가') -> 2번 등장
  ('오늘은', '기분이') -> 1번 등장
  ('오늘은', '일이') -> 1번 등장
  ('오늘은', '사람이') -> 1번 등장


## 5. 확률 계산하기

이제 통계를 바탕으로 확률을 계산합니다.
복잡한 공식 없이, 간단한 산수로 생각하면 됩니다.

### 확률 계산 방법
**확률 = (둘이 함께 나온 횟수) ÷ (앞 단어가 나온 전체 횟수)**

예를 들어:
1. "오늘은" 이라는 단어가 총 **5번** 나왔습니다.
2. 그 중 "오늘은 -> 날씨가" 패턴은 **2번** 나왔습니다.

그러면 확률은?
-> **2 나누기 5 = 0.4 (40%)** 가 됩니다.

In [5]:
print("[확률 계산 결과]")
print("앞 단어 -> 뒷 단어 : 확률")
print("=" * 30)

for (w1, w2), count in bigram_freq.items():
    # 확률 = (함께 나온 횟수) / (앞 단어 전체 횟수)
    prob = count / unigram_freq[w1]
    
    print(f"{w1} -> {w2} : {prob:.2f} ({prob*100:.0f}%) ")

[확률 계산 결과]
앞 단어 -> 뒷 단어 : 확률
오늘은 -> 날씨가 : 0.40 (40%) 
날씨가 -> 좋다 : 0.50 (50%) 
좋다 -> . : 1.00 (100%) 
. -> 오늘은 : 0.80 (80%) 
오늘은 -> 기분이 : 0.20 (20%) 
기분이 -> 좋다 : 1.00 (100%) 
오늘은 -> 일이 : 0.20 (20%) 
일이 -> 많다 : 1.00 (100%) 
많다 -> . : 1.00 (100%) 
오늘은 -> 사람이 : 0.20 (20%) 
사람이 -> 많다 : 1.00 (100%) 
날씨가 -> 맑다 : 0.50 (50%) 
맑다 -> . : 1.00 (100%) 


## 6. 모델 평가: 얼마나 헷갈려 할까? (PPL)

모델의 성능을 평가할 때는 **PPL(Perplexity, 혼란도)**라는 점수를 씁니다.

### 개념 시각화
PPL은 **"다음에 올 단어 후보의 개수"**라고 이해하면 쉽습니다.

**상황 A: 정답을 확신할 때 (좋은 상태)**
- "나는 학교에 [   ]"
- 후보: 간다 (확률 99%)
- **PPL = 1** (후보가 1개뿐이다!)

**상황 B: 헷갈릴 때 (나쁜 상태)**
- "나는 [   ]"
- 후보: 밥을, 학교에, 집에, 친구를... (후보가 100개)
- **PPL = 100** (후보가 100개나 되네...)

따라서 PPL 점수는 **낮을수록 (1에 가까울수록) 좋은 모델**입니다.

In [6]:
import math

def compute_bigram_perplexity(test_text, unigram_freq, bigram_freq):
    """
    PPL을 계산하는 함수입니다.
    결과값이 낮을수록 모델이 문장을 잘 이해한 것입니다.
    """
    test_tokens = nltk.word_tokenize(test_text)
    test_bigrams = list(ngrams(test_tokens, 2))

    N = len(test_bigrams)
    if N == 0: return 0

    log_prob_sum = 0

    for w1, w2 in test_bigrams:
        w1_count = unigram_freq.get(w1, 0)
        
        # 학습하지 않아서 모르는 단어가 나오면 아주 작은 확률을 줍니다.
        if w1_count == 0:
             prob = 1e-10
        else:
            prob = bigram_freq.get((w1, w2), 0) / w1_count
            
        if prob == 0:
            prob = 1e-10
            
        log_prob_sum += math.log2(prob)

    # 평균을 내고 2의 제곱을 해줍니다.
    cross_entropy = -log_prob_sum / N
    perplexity = math.pow(2, cross_entropy)

    return perplexity

In [7]:
test_sentences = [
    "오늘은 날씨가 좋다.",      # 학습했던 문장 그대로 (예상: 점수 좋음)
    "오늘은 기분이 좋다.",      # 배운 패턴 (예상: 점수 좋음)
    "기계 번역은 어렵다."        # 안 배운 단어들 (예상: 점수 나쁨)
]

print("[모델 평가]")
print("점수가 1에 가까울수록 좋습니다.")
print("-" * 40)

for sent in test_sentences:
    ppl = compute_bigram_perplexity(sent, unigram_freq, bigram_freq)
    
    # 결과 해석
    analysis = ""
    if ppl < 2: analysis = "[우수] AI: \"이건 확실히 아는 문장이야!\""
    elif ppl > 1000: analysis = "[실패] AI: \"모르는 단어 투성이야...\""
    
    print(f"문장: {sent:<15} | PPL 점수: {ppl:.2f}  {analysis}")

[모델 평가]
점수가 1에 가까울수록 좋습니다.
----------------------------------------
문장: 오늘은 날씨가 좋다.     | PPL 점수: 1.71  [우수] AI: "이건 확실히 아는 문장이야!"
문장: 오늘은 기분이 좋다.     | PPL 점수: 1.71  [우수] AI: "이건 확실히 아는 문장이야!"
문장: 기계 번역은 어렵다.     | PPL 점수: 10000000000.00  [실패] AI: "모르는 단어 투성이야..."


## 7. 핵심 요약 퀴즈

1. **Bigram 확률**을 구할 때 분모에 들어가는 값은 무엇인가요?
   - [ ] 앞 단어가 등장한 전체 횟수
   - [ ] 뒷 단어가 등장한 전체 횟수
   
2. 만약 **PPL 점수가 100점**이라면 무슨 뜻일까요?
   - [ ] 다음에 올 단어 후보가 1개뿐이라 확실하다.
   - [ ] 다음에 올 단어 후보가 100개쯤 되어서 매우 헷갈린다.

정답은 위에서 배운 내용을 다시 확인해보세요.